In [2]:
종로구 = [1111010100,1111010200,1111010300,1111010400,1111010500,1111010600,1111010700,1111010800,1111010900,1111011000,1111011100,1111011200,1111011300,1111011400,1111011500,1111011600,1111011700,1111011800,1111011900,1111012000,1111012100,1111012200,1111012300,1111012400,1111012500,1111012600,1111012700,1111012800,1111012900,1111013000,1111013100,1111013200,1111013300,1111013400,1111013500,1111013600,1111013700,1111013800,1111013900,1111014000,1111014100,1111014200,1111014300,1111014400,1111014500,1111014600,1111014700,1111014800,1111014900,1111015000,1111015100,1111015200,1111015300,1111015400,1111015500,1111015600,1111015700,1111015800,1111015900,1111016000,1111016100,1111016200,1111016300,1111016400,1111016500,1111016600,1111016700,1111016800,1111016900,1111017000,1111017100,1111017200,1111017300,1111017400,1111017500,1111017600,1111017700,1111017800,1111017900,1111018000,1111018100,1111018200,1111018300,1111018400,1111018500,1111018600,1111018700]
jongno = ['청운동','신교동','궁정동','효자동','창성동','통의동','적선동','통인동','누상동','누하동','옥인동','체부동','필운동','내자동','사직동','도렴동','당주동','내수동','세종로','신문로1가','신문로2가','청진동','서린동','수송동','중학동','종로1가','공평동','관훈동','견지동','와룡동','권농동','운니동','익선동','경운동','관철동','인사동','낙원동','종로2가','팔판동','삼청동','안국동','소격동','화동','사간동','송현동','가회동','재동','계동','원서동','훈정동','묘동','봉익동','돈의동','장사동','관수동','종로3가','인의동','예지동','원남동','연지동','종로4가','효제동','종로5가','종로6가','이화동','연건동','충신동','동숭동','혜화동','명륜1가','명륜2가','명륜4가','명륜3가','창신동','숭인동','교남동','평동','송월동','홍파동','교북동','행촌동','구기동','평창동','부암동','홍지동','신영동','무악동']

In [3]:
import requests #필요한 모듈들을 import 합니다.
from bs4 import BeautifulSoup
import pandas as pd

names = []
areas = []
prices = []
area_per_price = []
for area_code in 종로구:
    
    url = 'https://hogangnono.com/region/{}'.format(area_code) #url을 area_code를 이용하여 반복문으로 한번에 모두 가져옵니다.
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    name = []
    for pages in soup.findAll('li'):    # 아파트 이름을 가져옵니다
        for link in pages('h5',{'class':'name'}):
            data = link.text        
            name.append(data)
    area = []    
    for pages in soup.findAll('li'):    # 아프트 면적을 가져옵니다
        for link in pages('span',{'class':'area'}):
            data = link.text        
            area.append(data)
    price = []
    for pages in soup.findAll('li'):    #아파트 가겨을 가져옵니다
        for link in pages('span',{'class':'price'}):
            data = link.text        
            price.append(data)
            
    index_dash = [i for i, n in enumerate(price) if n == '-'] # 아파트 가격이 '-' 로 나와있는 아파트들의 index를 찾아냅니다
    for i in index_dash[::-1]: # '-'로 나와있는 아파트를 리스트에서 지웁니다
        del(name[i])
        del(area[i])
        del(price[i])
    
    new_name = [] # 아파트 이름 앞에 동 이름을 제거합니다
    for k in name:
        k_idx = k.find(' ')
        k = k[k_idx+1:]
        new_name.append(k)
        
    
    int_area = []
    for l in area: # 아파트 면적을 integer로 바꿔줍니다
        per = l[:-1]
        int_area.append(int(per))
    
    new_price = []
    for j in price: # 아파트 가격을 integer 로 바꿔줍니다

        if '억'in j:
            if len(j)>4:
                y_idx = j.find('억')
                pri = j[0:y_idx]
                pri1 = j[y_idx+1:]
                pri2 = pri1.replace(',','')
                j = ((int(pri)*10000)+(int(pri2)))*10000
                new_price.append(j)
            else:
                j = j.replace('억','')
                j = int(j)*100000000
                new_price.append(j)

        else:
            j = j.replace(',','')
            j = int(j)*10000
            new_price.append(j)
            
    per_price = [] # 평당 가격을 계산합니다
    for a in range(len(new_price)):
        perp = new_price[a]/int_area[a]
        per_price.append(round(perp))
        
    
    
    names.append(new_name) # lists in lists 로 저장합니다
    areas.append(int_area)
    prices.append(new_price)
    area_per_price.append(per_price)
    
amean = []

for i in range(len(jongno)): # 동별로 평당가격의 mean을 구해줍니다
    if len(area_per_price[i])>0:
        mean = sum(area_per_price[i])/len(area_per_price[i])
        amean.append(round(mean))
        
    else:
        amean.append(0)

kk = {'dong':종로구,'per price':amean}
df = pd.DataFrame(kk,index = jongno)
df

,dong,per price
청운동,1111010100,21491478
신교동,1111010200,0
궁정동,1111010300,0
효자동,1111010400,0
창성동,1111010500,0
통의동,1111010600,0
적선동,1111010700,11153846
통인동,1111010800,0
누상동,1111010900,0
누하동,1111011000,0


In [4]:
df.to_csv('jongnogu.csv')